<a href="https://colab.research.google.com/github/rooksandkings/APIs/blob/master/Melchior_Technical_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
!{sys.executable} -m pip install --upgrade setuptools
# !{sys.executable} -m pip install -r requirements.txt

     |████████████████████████████████| 583kB 6.5MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 42.0.1
    Uninstalling setuptools-42.0.1:
      Successfully uninstalled setuptools-42.0.1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Clean data in case it is already in memory:
data = None

path = "/content/drive/My Drive/Colab Notebooks/SpyDaily.csv"
data = pd.read_csv(path)
# Dataset is now stored in a Pandas Dataframe

print("Imported Data:")
data.head()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Imported Data:


,Date,Time,Open,High,Low,Last,Volume,NumberOfTrades,BidVolume,AskVolume
0,2007/10/15,00:00:00,156.27,156.36,153.94,155.01,161115008,0,0,0
1,2007/10/16,00:00:00,154.41,156.48,153.47,153.78,166454496,0,0,0
2,2007/10/17,00:00:00,154.98,155.09,152.47,154.25,216672800,0,0,0
3,2007/10/18,00:00:00,153.45,154.19,153.08,153.69,148447904,0,0,0
4,2007/10/19,00:00:00,153.09,156.48,149.66,149.67,297151008,0,0,0


In [3]:
# Cleaning Data:
# Removing Time, NumberOfTrades, BidVolume, AskVolume from dataframe

data = data.drop(columns = " Time", errors='ignore')
data = data.drop(columns = " NumberOfTrades", errors='ignore')
data = data.drop(columns = " BidVolume", errors='ignore')
data = data.drop(columns = " AskVolume", errors='ignore')

data = data.rename(columns={' Open': 'Open', ' High': 'High', ' Low': 'Low', ' Last': 'Close', ' Volume': 'Volume'})

# list(data.columns)

print("Data after Column Rename / Drop of unneeded Columns")    
data.head()

Data after Column Rename / Drop of unneeded Columns


,Date,Open,High,Low,Close,Volume
0,2007/10/15,156.27,156.36,153.94,155.01,161115008
1,2007/10/16,154.41,156.48,153.47,153.78,166454496
2,2007/10/17,154.98,155.09,152.47,154.25,216672800
3,2007/10/18,153.45,154.19,153.08,153.69,148447904
4,2007/10/19,153.09,156.48,149.66,149.67,297151008


In [4]:
import scipy.stats as st
from sklearn.preprocessing import LabelEncoder

# Inputs for Keras

# Settings for Local Window Variables
ShWindow = 5
LgWindow = 25
ZscoreWindow = 25

# (1) Candlestick and Volume characteristics:
# Candle Length, Candle Body Length, Body % size of Upper Tail, Body % size of Lower Tail, Upper Tail % of Candle Length, 
# Lower Tail % of Candle Length, Body Size Stdev from mean 5, 25 day, Upper/Lower Tail size Stdev from mean 5 day, 25 day+
data['LengthofCandle'] = data['High'] - data['Low']
data['LengthofCandleBody'] = (data['Open'] - data['Close']).abs()

data['LowerTailLength'] = data[["Open", "Close"]].min(axis=1)-data['Low']
data['UpperTailLength'] = data['High'] - data[["Open", "Close"]].max(axis=1)

data['CandleLengthShStdev'] = data['LengthofCandle'].rolling(min_periods=ShWindow, window=ShWindow).std()
data['CandleLengthLgStdev'] = data['LengthofCandle'].rolling(min_periods=LgWindow, window=LgWindow).std()
data['CandleLengthShVLgStdev'] = data['CandleLengthShStdev'] / data['CandleLengthLgStdev']

data['CandleBodyLengthShStdev'] = data['LengthofCandleBody'].rolling(min_periods=ShWindow, window=ShWindow).std()
data['CandleBodyLengthLgStdev'] = data['LengthofCandleBody'].rolling(min_periods=LgWindow, window=LgWindow).std()
data['CandleBodyLengthShVLgStdev'] = data['CandleBodyLengthShStdev'] / data['CandleBodyLengthLgStdev']

data['LowerTailShStDev'] = data['LowerTailLength'].rolling(min_periods=ShWindow, window=ShWindow).std()
data['LowerTailLgStDev'] = data['LowerTailLength'].rolling(min_periods=LgWindow, window=LgWindow).std()
data['LowerTailShVLgStdev'] = data['LowerTailShStDev'] / data['LowerTailLgStDev']

data['UpperTailShStDev'] = data['UpperTailLength'].rolling(min_periods=ShWindow, window=ShWindow).std()
data['UpperTailLgStDev'] = data['UpperTailLength'].rolling(min_periods=LgWindow, window=LgWindow).std()
data['UpperTailShVLgStdev'] = data['UpperTailShStDev'] / data['UpperTailLgStDev']

# data['VolShStDev'] = data['Volume'].rolling(min_periods=ShWindow, window=ShWindow).std()
# data['VolLgStDev'] = data['Volume'].rolling(min_periods=LgWindow, window=LgWindow).std()

# The following was removed due to LoCB, LoC = small number or 0, producing large and infinity ratios. They were replaced with Pvalues
# data['LowerTailPctCandleBody'] = data['LowerTailLength'] / data['LengthofCandleBody']
# data['UpperTailPctCandleBody'] = data['UpperTailLength'] / data['LengthofCandleBody']
# data['LowerTailPctCandleLength'] = data['LowerTailLength'] / data['LengthofCandle']
# data['UpperTailPctCandleLength'] = data['UpperTailLength'] / data['LengthofCandle']

data['CandleLengthLgPValue'] = st.norm.cdf((data['LengthofCandle'] - data['LengthofCandle'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['LengthofCandle'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std())
data['CandleBodyLengthLgPValue'] = st.norm.cdf((data['LengthofCandleBody'] - data['LengthofCandleBody'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['LengthofCandleBody'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std())
data['UpperTailLgPValue'] = st.norm.cdf((data['UpperTailLength'] - data['UpperTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['UpperTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std())
data['LowerTailLgPValue'] = st.norm.cdf((data['LowerTailLength'] - data['LowerTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['LowerTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std())
data['VolLgPValue'] = st.norm.cdf((data['Volume'] - data['Volume'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['Volume'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std())

# This will produce errors due to st.norm.cdf operating on NaN due to LgWindow. The error is ignorable / can be avoided if the LG window is cut before this operation.
# /usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
#   return (a < x) & (x < b)
# /usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
#   return (a < x) & (x < b)
# /usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
#   cond2 = (x >= _b) & cond0

# (2) Moving Average:
# Difference from Short moving average (Open, High, Low, Close, Avg(Open, High, Low, Close))
data['DiffShortO_MA'] = data['Open']-data['Open'].rolling(min_periods=ShWindow, window=ShWindow).mean()
data['DiffShortH_MA'] = data['High']-data['High'].rolling(min_periods=ShWindow, window=ShWindow).mean()
data['DiffShortL_MA'] = data['Low']-data['Low'].rolling(min_periods=ShWindow, window=ShWindow).mean()
data['DiffShortC_MA'] = data['Close']-data['Close'].rolling(min_periods=ShWindow, window=ShWindow).mean()
data['DiffShortA_MA'] = (data['DiffShortO_MA'] + data['DiffShortH_MA'] + data['DiffShortL_MA'] + data['DiffShortC_MA']) / 4
    
# (3) Difference from Long moving average (Open, High, Low, Close, Avg(Open, High, Low, Close))
data['DiffLongO_MA'] = data['Open']-data['Open'].rolling(min_periods=LgWindow, window=LgWindow).mean()
data['DiffLongH_MA'] = data['High']-data['High'].rolling(min_periods=LgWindow, window=LgWindow).mean()
data['DiffLongL_MA'] = data['Low']-data['Low'].rolling(min_periods=LgWindow, window=LgWindow).mean()
data['DiffLongC_MA'] = data['Close']-data['Close'].rolling(min_periods=LgWindow, window=LgWindow).mean()
data['DiffLongA_MA'] = (data['DiffLongO_MA'] + data['DiffLongH_MA'] + data['DiffLongL_MA'] + data['DiffLongC_MA']) / 4

# (4) Slope of 5, 25 day moving average
data['5DSlpOMA'] = (data['Open'] - data['Open'].shift(5)) / 5
data['25DSlpOMA'] = (data['Open'] - data['Open'].shift(25)) / 25
# May need some adjustments here? MA vs Z-Score MA?

# (5) Slope of average of Open, High, Low, Close 3 to 5, 4-6, 5-7, 6-8 days prior
data['3-5DSlpOMA'] = data['Open'].shift(3).diff(periods=2) / 3
data['4-6DSlpOMA'] = data['Open'].shift(4).diff(periods=2) / 3
data['5-7DSlpOMA'] = data['Open'].shift(5).diff(periods=2) / 3
data['6-8DSlpOMA'] = data['Open'].shift(6).diff(periods=2) / 3
# Needs to be checked

# (6) Relative Position:
# High of Day (HOD) Higher than HOD, Open, Close, Low of Day (LOD) Previous 1-5 days?
data['HODvHOD-1'] = data['High'] - data['High'].shift(1)
data['HODvHOD-2'] = data['High'] - data['High'].shift(2)
data['HODvHOD-3'] = data['High'] - data['High'].shift(3)
data['HODvHOD-4'] = data['High'] - data['High'].shift(4)
data['HODvHOD-5'] = data['High'] - data['High'].shift(5)

data['HODvO-1'] = data['High'] - data['Open'].shift(1)
data['HODvO-2'] = data['High'] - data['Open'].shift(2)
data['HODvO-3'] = data['High'] - data['Open'].shift(3)
data['HODvO-4'] = data['High'] - data['Open'].shift(4)
data['HODvO-5'] = data['High'] - data['Open'].shift(5)

data['HODvC-1'] = data['High'] - data['Close'].shift(1)
data['HODvC-2'] = data['High'] - data['Close'].shift(2)
data['HODvC-3'] = data['High'] - data['Close'].shift(3)
data['HODvC-4'] = data['High'] - data['Close'].shift(4)
data['HODvC-5'] = data['High'] - data['Close'].shift(5)

data['HODvLOD-1'] = data['High'] - data['Low'].shift(1)
data['HODvLOD-2'] = data['High'] - data['Low'].shift(2)
data['HODvLOD-3'] = data['High'] - data['Low'].shift(3)
data['HODvLOD-4'] = data['High'] - data['Low'].shift(4)
data['HODvLOD-5'] = data['High'] - data['Low'].shift(5)

# Open Higher than Open, Close, LOD Previous 1-5 days?
data['OvO-1'] = data['Open'] - data['Open'].shift(1)
data['OvO-2'] = data['Open'] - data['Open'].shift(2)
data['OvO-3'] = data['Open'] - data['Open'].shift(3)
data['OvO-4'] = data['Open'] - data['Open'].shift(4)
data['OvO-5'] = data['Open'] - data['Open'].shift(5)

data['OvC-1'] = data['Open'] - data['Close'].shift(1)
data['OvC-2'] = data['Open'] - data['Close'].shift(2)
data['OvC-3'] = data['Open'] - data['Close'].shift(3)
data['OvC-4'] = data['Open'] - data['Close'].shift(4)
data['OvC-5'] = data['Open'] - data['Close'].shift(5)

data['OvLOD-1'] = data['Open'] - data['Low'].shift(1)
data['OvLOD-2'] = data['Open'] - data['Low'].shift(2)
data['OvLOD-3'] = data['Open'] - data['Low'].shift(3)
data['OvLOD-4'] = data['Open'] - data['Low'].shift(4)
data['OvLOD-5'] = data['Open'] - data['Low'].shift(5)

# Close Higher than Close, LOD, previous 1-5 days?
data['CvC-1'] = data['Close'] - data['Close'].shift(1)
data['CvC-2'] = data['Close'] - data['Close'].shift(2)
data['CvC-3'] = data['Close'] - data['Close'].shift(3)
data['CvC-4'] = data['Close'] - data['Close'].shift(4)
data['CvC-5'] = data['Close'] - data['Close'].shift(5)

data['CvC-1'] = data['Close'] - data['Low'].shift(1)
data['CvC-2'] = data['Close'] - data['Low'].shift(2)
data['CvC-3'] = data['Close'] - data['Low'].shift(3)
data['CvC-4'] = data['Close'] - data['Low'].shift(4)
data['CvC-5'] = data['Close'] - data['Low'].shift(5)

# Lod Higher than LOD previous 1-5 days?
data['LODvLOD-1'] = data['Low'] - data['Low'].shift(1)
data['LODvLOD-2'] = data['Low'] - data['Low'].shift(2)
data['LODvLOD-3'] = data['Low'] - data['Low'].shift(3)
data['LODvLOD-4'] = data['Low'] - data['Low'].shift(4)
data['LODvLOD-5'] = data['Low'] - data['Low'].shift(5)

# (7) Engulfing:
# Body, Body and Tail, Tail engulfing previous 1-5 days? Outside bounds of Body, Body and Tail of previous 1-5 Days?
# Day(s) 5 v 4, 5 v 3, 5 v 2, 5 v 1, 4 v 3, 4 v 2, 4 v 1, 3 v 2, 3 v 1, 2 v 1
# & astype(float) to one-hot encode
# One Hot Encoding reversal not needed if reversal can be deduced from former derivation: e.g. If .shift(5) > .shift(4), it can be deduced that .shift(4) < .shift(5) - relevant to engulfing

data['EngulfingBodyvBody5v4'] = ((data['Open'].shift(5) >= data['Open'].shift(4)) & (data['Close'].shift(5) <= data['Close'].shift(4))).astype(int)
data['EngulfingBodyvBody5v3'] = ((data['Open'].shift(5) >= data['Open'].shift(3)) & (data['Close'].shift(5) <= data['Close'].shift(3))).astype(int)
data['EngulfingBodyvBody5v2'] = ((data['Open'].shift(5) >= data['Open'].shift(2)) & (data['Close'].shift(5) <= data['Close'].shift(2))).astype(int)
data['EngulfingBodyvBody5v1'] = ((data['Open'].shift(5) >= data['Open'].shift(1)) & (data['Close'].shift(5) <= data['Close'].shift(1))).astype(int)
data['EngulfingBodyvBody4v3'] = ((data['Open'].shift(4) >= data['Open'].shift(3)) & (data['Close'].shift(4) <= data['Close'].shift(3))).astype(int)
data['EngulfingBodyvBody4v2'] = ((data['Open'].shift(4) >= data['Open'].shift(2)) & (data['Close'].shift(4) <= data['Close'].shift(2))).astype(int)
data['EngulfingBodyvBody4v1'] = ((data['Open'].shift(4) >= data['Open'].shift(1)) & (data['Close'].shift(4) <= data['Close'].shift(1))).astype(int)
data['EngulfingBodyvBody3v2'] = ((data['Open'].shift(3) >= data['Open'].shift(2)) & (data['Close'].shift(3) <= data['Close'].shift(2))).astype(int)
data['EngulfingBodyvBody3v1'] = ((data['Open'].shift(3) >= data['Open'].shift(1)) & (data['Close'].shift(3) <= data['Close'].shift(1))).astype(int)
data['EngulfingBodyvBody2v1'] = ((data['Open'].shift(2) >= data['Open'].shift(1)) & (data['Close'].shift(2) <= data['Close'].shift(1))).astype(int)

data['EngulfingCandleLengthvCandleLength5v4'] = ((data['High'].shift(5) >= data['High'].shift(4)) & (data['Low'].shift(5) <= data['Low'].shift(4))).astype(int)
data['EngulfingCandleLengthvCandleLength5v3'] = ((data['High'].shift(5) >= data['High'].shift(3)) & (data['Low'].shift(5) <= data['Low'].shift(3))).astype(int)
data['EngulfingCandleLengthvCandleLength5v2'] = ((data['High'].shift(5) >= data['High'].shift(2)) & (data['Low'].shift(5) <= data['Low'].shift(2))).astype(int)
data['EngulfingCandleLengthvCandleLength5v4'] = ((data['High'].shift(5) >= data['High'].shift(1)) & (data['Low'].shift(5) <= data['Low'].shift(1))).astype(int)
data['EngulfingCandleLengthvCandleLength4v3'] = ((data['High'].shift(4) >= data['High'].shift(3)) & (data['Low'].shift(4) <= data['Low'].shift(3))).astype(int)
data['EngulfingCandleLengthvCandleLength3v2'] = ((data['High'].shift(4) >= data['High'].shift(2)) & (data['Low'].shift(4) <= data['Low'].shift(2))).astype(int)
data['EngulfingCandleLengthvCandleLength2v1'] = ((data['High'].shift(4) >= data['High'].shift(1)) & (data['Low'].shift(4) <= data['Low'].shift(1))).astype(int)
data['EngulfingCandleLengthvCandleLength3v2'] = ((data['High'].shift(3) >= data['High'].shift(2)) & (data['Low'].shift(3) <= data['Low'].shift(2))).astype(int)
data['EngulfingCandleLengthvCandleLength3v1'] = ((data['High'].shift(3) >= data['High'].shift(1)) & (data['Low'].shift(3) <= data['Low'].shift(1))).astype(int)
data['EngulfingCandleLengthvCandleLength2v1'] = ((data['High'].shift(2) >= data['High'].shift(1)) & (data['Low'].shift(2) <= data['Low'].shift(1))).astype(int)

# (8) Stars or Inverse Stars:
# Tailess if Z score < 1 (9.2%)
Tailless_ZScore_Cap = -1
data['Lower-Tailless'] = ((data['LowerTailLength'] - data['LowerTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['LowerTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std() < Tailless_ZScore_Cap).astype(int)
data['Upper-Tailless'] = ((data['UpperTailLength'] - data['UpperTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['UpperTailLength'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std() < Tailless_ZScore_Cap).astype(int)

# Bodyless if Z score < -1 (9.2%)
Bodyless_ZScore_Cap = -1
data['Bodyless'] = ((data['LengthofCandleBody'] - data['LengthofCandleBody'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).mean()) / data['LengthofCandleBody'].rolling(min_periods=ZscoreWindow, window=ZscoreWindow).std() < Bodyless_ZScore_Cap ).astype(int)

# (9) Positioning:
# Gap(s)?
data['GapPrevious1Day'] = ((data['Open'] > data['High'].shift(1)) | (data['Open'] < data['Low'].shift(1))).astype(int)
data['GapPrevious2Day'] = ((data['Open'].shift(1) > data['High'].shift(2)) | (data['Open'].shift(1) < data['Low'].shift(2))).astype(int)
data['GapPrevious3Day'] = ((data['Open'].shift(2) > data['High'].shift(3)) | (data['Open'].shift(2) < data['Low'].shift(3))).astype(int)
data['GapPrevious4Day'] = ((data['Open'].shift(3) > data['High'].shift(4)) | (data['Open'].shift(3) < data['Low'].shift(4))).astype(int)
data['GapPrevious5Day'] = ((data['Open'].shift(4) > data['High'].shift(5)) | (data['Open'].shift(4) < data['Low'].shift(5))).astype(int)

# Close Higher than previous 1-5 days?, Open higher than previous 1-5 days? Low of day Higher than 
data['CloseHigherv1'] = (data['Close'] > data['Close'].shift(1)).astype(int)
data['CloseHigherv2'] = (data['Close'].shift(1) > data['Close'].shift(2)).astype(int)
data['CloseHigherv3'] = (data['Close'].shift(2) > data['Close'].shift(3)).astype(int)
data['CloseHigherv4'] = (data['Close'].shift(3) > data['Close'].shift(4)).astype(int)
data['CloseHigherv5'] = (data['Close'].shift(4) > data['Close'].shift(5)).astype(int)

data['OpenHigherv1'] = (data['Open'] > data['Open'].shift(1)).astype(int)
data['OpenHigherv2'] = (data['Open'].shift(1) > data['Open'].shift(2)).astype(int)
data['OpenHigherv3'] = (data['Open'].shift(2) > data['Open'].shift(3)).astype(int)
data['OpenHigherv4'] = (data['Open'].shift(3) > data['Open'].shift(4)).astype(int)
data['OpenHigherv5'] = (data['Open'].shift(4) > data['Open'].shift(5)).astype(int)

data['LodHigherv1'] = (data['Low'] > data['Low'].shift(1)).astype(int)
data['LodHigherv2'] = (data['Low'].shift(1) > data['Low'].shift(2)).astype(int)
data['LodHigherv3'] = (data['Low'].shift(2) > data['Low'].shift(3)).astype(int)
data['LodHigherv4'] = (data['Low'].shift(3) > data['Low'].shift(4)).astype(int)
data['LodHigherv5'] = (data['Low'].shift(4) > data['Low'].shift(5)).astype(int)

data['HodHigherv1'] = (data['High'] > data['High'].shift(1)).astype(int)
data['HodHigherv2'] = (data['High'].shift(1) > data['High'].shift(2)).astype(int)
data['HodHigherv3'] = (data['High'].shift(2) > data['High'].shift(3)).astype(int)
data['HodHigherv4'] = (data['High'].shift(3) > data['High'].shift(4)).astype(int)
data['HodHigherv5'] = (data['High'].shift(4) > data['High'].shift(5)).astype(int)

# (10) One Hot Encoding Day of Week
# Clear past iterations:

data = data.drop(columns = "Monday", errors='ignore')
data = data.drop(columns = "Tuesday", errors='ignore')
data = data.drop(columns = "Wednesday", errors='ignore')
data = data.drop(columns = "Thursday", errors='ignore')
data = data.drop(columns = "Friday", errors='ignore')

# Convert Day to Day Name
data['DayOfWeek'] = (pd.to_datetime(data['Date'], errors='coerce')).dt.day_name()

# Concat One Hot Encode With original Dataframe 
data = pd.concat((data, pd.get_dummies(data['DayOfWeek'])), axis=1)
data.drop(['DayOfWeek'],axis=1, inplace=True)

# Queries
# data.loc[data['GapPrevious1Day'] == 1]
# data[['Date', 'Open', 'High', 'Low', 'Close', 'LengthofCandle', 'LengthofCandleBody', 'UpperTailPctCandleBody', 'LowerTailPctCandleBody', 'UpperTailPctCandleLength', 'LowerTailPctCandleLength', 'UpperTailPctLowerTail', 'CandleLength5DStdev']].head(30)
# data[['CandleLengthLgPValue']].head(30)
# data[['Open']].shift(1).head(50)
# data[['Open']].head(50)
# data[['Wednesday']].head(50)
# print(Tailless_ZScore_Cap)
# data.nsmallest(30, 'High-Tailless')
# data.head(30)

# data[['Date', 'Open', 'High', 'Low', 'Close', 'Upper-Tailless', 'Lower-Tailless']].head(50)

# Do we need to note how many times the slope inverts over a rolling time frame?
# ***Pivot Points have not been implemented yet***
# Day of week not implemented
# Don't Forget to remove 25 days of data from training due to 25 Day SMA NaN!

/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1807: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= _b) & cond0


In [0]:
# Outputs for Keras 
# ***DO NOT USE AS INPUTS! INCLUDES FUTURE VALUES!***

# Notes: A variation should be tried using .round()

# Open Standard Dev Tmp (For one hot encoding)
data['OpenvOpen_CL_ShStdevTmp'] = (data['Open'].shift(-1) - data['Open']) / data['CandleLengthShStdev']
# data['OpenvOpen_CB_ShStdevTmp'] = (data['Open'].shift(-1) - data['Open']) / data['CandleBodyLengthShStdev']
# data['OpenvOpen_UT_ShStdevTmp'] = (data['Open'].shift(-1) - data['Open']) / data['UpperTailShStDev']
# data['OpenvOpen_LT_ShStdevTmp'] = (data['Open'].shift(-1) - data['Open']) / data['LowerTailShStDev']

# data['OpenvOpen_CL_LgStdevTmp'] = (data['Open'].shift(-1) - data['Open']) / data['CandleLengthLgStdev']
# data['OpenvOpen_CB_LgStdevTmp'] = (data['Open'].shift(-1) - data['Open']) / data['CandleBodyLengthLgStdev']
# data['OpenvOpen_UT_LgStdevTmp'] = (data['Open'].shift(-1) - data['Open']) / data['UpperTailLgStDev']
# data['OpenvOpen_LT_LgStdevTmp'] = (data['Open'].shift(-1) - data['Open']) / data['LowerTailLgStDev']

# data['OpenvHigh_CL_ShStdevTmp'] = (data['High'].shift(-1) - data['Open']) / data['CandleLengthShStdev']
# data['OpenvHigh_CB_ShStdevTmp'] = (data['High'].shift(-1) - data['Open']) / data['CandleBodyLengthShStdev']
# data['OpenvHigh_UT_ShStdevTmp'] = (data['High'].shift(-1) - data['Open']) / data['UpperTailShStDev']
# data['OpenvHigh_LT_ShStdevTmp'] = (data['High'].shift(-1) - data['Open']) / data['LowerTailShStDev']

# data['OpenvHigh_CL_LgStdevTmp'] = (data['High'].shift(-1) - data['Open']) / data['CandleLengthLgStdev']
# data['OpenvHigh_CB_LgStdevTmp'] = (data['High'].shift(-1) - data['Open']) / data['CandleBodyLengthLgStdev']
# data['OpenvHigh_UT_LgStdevTmp'] = (data['High'].shift(-1) - data['Open']) / data['UpperTailLgStDev']
# data['OpenvHigh_LT_LgStdevTmp'] = (data['High'].shift(-1) - data['Open']) / data['LowerTailLgStDev']

# data['OpenvLow_CL_ShStdevTmp'] = (data['Low'].shift(-1) - data['Open']) / data['CandleLengthShStdev']
# data['OpenvLow_CB_ShStdevTmp'] = (data['Low'].shift(-1) - data['Open']) / data['CandleBodyLengthShStdev']
# data['OpenvLow_UT_ShStdevTmp'] = (data['Low'].shift(-1) - data['Open']) / data['UpperTailShStDev']
# data['OpenvLow_LT_ShStdevTmp'] = (data['Low'].shift(-1) - data['Open']) / data['LowerTailShStDev']

# data['OpenvLow_CL_LgStdevTmp'] = (data['Low'].shift(-1) - data['Open']) / data['CandleLengthLgStdev']
# data['OpenvLow_CB_LgStdevTmp'] = (data['Low'].shift(-1) - data['Open']) / data['CandleBodyLengthLgStdev']
# data['OpenvLow_UT_LgStdevTmp'] = (data['Low'].shift(-1) - data['Open']) / data['UpperTailLgStDev']
# data['OpenvLow_LT_LgStdevTmp'] = (data['Low'].shift(-1) - data['Open']) / data['LowerTailLgStDev']

# data['OpenvClose_CL_ShStdevTmp'] = (data['Close'].shift(-1) - data['Open']) / data['CandleLengthShStdev']
# data['OpenvClose_CB_ShStdevTmp'] = (data['Close'].shift(-1) - data['Open']) / data['CandleBodyLengthShStdev']
# data['OpenvClose_UT_ShStdevTmp'] = (data['Close'].shift(-1) - data['Open']) / data['UpperTailShStDev']
# data['OpenvClose_LT_ShStdevTmp'] = (data['Close'].shift(-1) - data['Open']) / data['LowerTailShStDev']

# data['OpenvClose_CL_LgStdevTmp'] = (data['Close'].shift(-1) - data['Open']) / data['CandleLengthLgStdev']
# data['OpenvClose_CB_LgStdevTmp'] = (data['Close'].shift(-1) - data['Open']) / data['CandleBodyLengthLgStdev']
# data['OpenvClose_UT_LgStdevTmp'] = (data['Close'].shift(-1) - data['Open']) / data['UpperTailLgStDev']
# data['OpenvClose_LT_LgStdevTmp'] = (data['Close'].shift(-1) - data['Open']) / data['LowerTailLgStDev']

# # High Standard Dev Tmp (For one hot encoding)
# data['HighvOpen_CL_ShStdevTmp'] = (data['Open'].shift(-1) - data['High']) / data['CandleLengthShStdev']
# data['HighvOpen_CB_ShStdevTmp'] = (data['Open'].shift(-1) - data['High']) / data['CandleBodyLengthShStdev']
# data['HighvOpen_UT_ShStdevTmp'] = (data['Open'].shift(-1) - data['High']) / data['UpperTailShStDev']
# data['HighvOpen_LT_ShStdevTmp'] = (data['Open'].shift(-1) - data['High']) / data['LowerTailShStDev']

# data['HighvOpen_CL_LgStdevTmp'] = (data['Open'].shift(-1) - data['High']) / data['CandleLengthLgStdev']
# data['HighvOpen_CB_LgStdevTmp'] = (data['Open'].shift(-1) - data['High']) / data['CandleBodyLengthLgStdev']
# data['HighvOpen_UT_LgStdevTmp'] = (data['Open'].shift(-1) - data['High']) / data['UpperTailLgStDev']
# data['HighvOpen_LT_LgStdevTmp'] = (data['Open'].shift(-1) - data['High']) / data['LowerTailLgStDev']

# data['HighvHigh_CL_ShStdevTmp'] = (data['High'].shift(-1) - data['High']) / data['CandleLengthShStdev']
# data['HighvHigh_CB_ShStdevTmp'] = (data['High'].shift(-1) - data['High']) / data['CandleBodyLengthShStdev']
# data['HighvHigh_UT_ShStdevTmp'] = (data['High'].shift(-1) - data['High']) / data['UpperTailShStDev']
# data['HighvHigh_LT_ShStdevTmp'] = (data['High'].shift(-1) - data['High']) / data['LowerTailShStDev']

# data['HighvHigh_CL_LgStdevTmp'] = (data['High'].shift(-1) - data['High']) / data['CandleLengthLgStdev']
# data['HighvHigh_CB_LgStdevTmp'] = (data['High'].shift(-1) - data['High']) / data['CandleBodyLengthLgStdev']
# data['HighvHigh_UT_LgStdevTmp'] = (data['High'].shift(-1) - data['High']) / data['UpperTailLgStDev']
# data['HighvHigh_LT_LgStdevTmp'] = (data['High'].shift(-1) - data['High']) / data['LowerTailLgStDev']

# data['HighvLow_CL_ShStdevTmp'] = (data['Low'].shift(-1) - data['High']) / data['CandleLengthShStdev']
# data['HighvLow_CB_ShStdevTmp'] = (data['Low'].shift(-1) - data['High']) / data['CandleBodyLengthShStdev']
# data['HighvLow_UT_ShStdevTmp'] = (data['Low'].shift(-1) - data['High']) / data['UpperTailShStDev']
# data['HighvLow_LT_ShStdevTmp'] = (data['Low'].shift(-1) - data['High']) / data['LowerTailShStDev']

# data['HighvLow_CL_LgStdevTmp'] = (data['Low'].shift(-1) - data['High']) / data['CandleLengthLgStdev']
# data['HighvLow_CB_LgStdevTmp'] = (data['Low'].shift(-1) - data['High']) / data['CandleBodyLengthLgStdev']
# data['HighvLow_UT_LgStdevTmp'] = (data['Low'].shift(-1) - data['High']) / data['UpperTailLgStDev']
# data['HighvLow_LT_LgStdevTmp'] = (data['Low'].shift(-1) - data['High']) / data['LowerTailLgStDev']

# data['HighvClose_CL_ShStdevTmp'] = (data['Close'].shift(-1) - data['High']) / data['CandleLengthShStdev']
# data['HighvClose_CB_ShStdevTmp'] = (data['Close'].shift(-1) - data['High']) / data['CandleBodyLengthShStdev']
# data['HighvClose_UT_ShStdevTmp'] = (data['Close'].shift(-1) - data['High']) / data['UpperTailShStDev']
# data['HighvClose_LT_ShStdevTmp'] = (data['Close'].shift(-1) - data['High']) / data['LowerTailShStDev']

# data['HighvClose_CL_LgStdevTmp'] = (data['Close'].shift(-1) - data['High']) / data['CandleLengthLgStdev']
# data['HighvClose_CB_LgStdevTmp'] = (data['Close'].shift(-1) - data['High']) / data['CandleBodyLengthLgStdev']
# data['HighvClose_UT_LgStdevTmp'] = (data['Close'].shift(-1) - data['High']) / data['UpperTailLgStDev']
# data['HighvClose_LT_LgStdevTmp'] = (data['Close'].shift(-1) - data['High']) / data['LowerTailLgStDev']

# # Low Standard Dev Tmp (For one hot encoding)
# data['LowvOpen_CL_ShStdevTmp'] = (data['Open'].shift(-1) - data['Low']) / data['CandleLengthShStdev']
# data['LowvOpen_CB_ShStdevTmp'] = (data['Open'].shift(-1) - data['Low']) / data['CandleBodyLengthShStdev']
# data['LowvOpen_UT_ShStdevTmp'] = (data['Open'].shift(-1) - data['Low']) / data['UpperTailShStDev']
# data['LowvOpen_LT_ShStdevTmp'] = (data['Open'].shift(-1) - data['Low']) / data['LowerTailShStDev']

# data['LowvOpen_CL_LgStdevTmp'] = (data['Open'].shift(-1) - data['Low']) / data['CandleLengthLgStdev']
# data['LowvOpen_CB_LgStdevTmp'] = (data['Open'].shift(-1) - data['Low']) / data['CandleBodyLengthLgStdev']
# data['LowvOpen_UT_LgStdevTmp'] = (data['Open'].shift(-1) - data['Low']) / data['UpperTailLgStDev']
# data['LowvOpen_LT_LgStdevTmp'] = (data['Open'].shift(-1) - data['Low']) / data['LowerTailLgStDev']

# data['LowvHigh_CL_ShStdevTmp'] = (data['High'].shift(-1) - data['Low']) / data['CandleLengthShStdev']
# data['LowvHigh_CB_ShStdevTmp'] = (data['High'].shift(-1) - data['Low']) / data['CandleBodyLengthShStdev']
# data['LowvHigh_UT_ShStdevTmp'] = (data['High'].shift(-1) - data['Low']) / data['UpperTailShStDev']
# data['LowvHigh_LT_ShStdevTmp'] = (data['High'].shift(-1) - data['Low']) / data['LowerTailShStDev']

# data['LowvHigh_CL_LgStdevTmp'] = (data['High'].shift(-1) - data['Low']) / data['CandleLengthLgStdev']
# data['LowvHigh_CB_LgStdevTmp'] = (data['High'].shift(-1) - data['Low']) / data['CandleBodyLengthLgStdev']
# data['LowvHigh_UT_LgStdevTmp'] = (data['High'].shift(-1) - data['Low']) / data['UpperTailLgStDev']
# data['LowvHigh_LT_LgStdevTmp'] = (data['High'].shift(-1) - data['Low']) / data['LowerTailLgStDev']

# data['LowvLow_CL_ShStdevTmp'] = (data['Low'].shift(-1) - data['Low']) / data['CandleLengthShStdev']
# data['LowvLow_CB_ShStdevTmp'] = (data['Low'].shift(-1) - data['Low']) / data['CandleBodyLengthShStdev']
# data['LowvLow_UT_ShStdevTmp'] = (data['Low'].shift(-1) - data['Low']) / data['UpperTailShStDev']
# data['LowvLow_LT_ShStdevTmp'] = (data['Low'].shift(-1) - data['Low']) / data['LowerTailShStDev']

# data['LowvLow_CL_LgStdevTmp'] = (data['Low'].shift(-1) - data['Low']) / data['CandleLengthLgStdev']
# data['LowvLow_CB_LgStdevTmp'] = (data['Low'].shift(-1) - data['Low']) / data['CandleBodyLengthLgStdev']
# data['LowvLow_UT_LgStdevTmp'] = (data['Low'].shift(-1) - data['Low']) / data['UpperTailLgStDev']
# data['LowvLow_LT_LgStdevTmp'] = (data['Low'].shift(-1) - data['Low']) / data['LowerTailLgStDev']

# data['LowvClose_CL_ShStdevTmp'] = (data['Close'].shift(-1) - data['Low']) / data['CandleLengthShStdev']
# data['LowvClose_CB_ShStdevTmp'] = (data['Close'].shift(-1) - data['Low']) / data['CandleBodyLengthShStdev']
# data['LowvClose_UT_ShStdevTmp'] = (data['Close'].shift(-1) - data['Low']) / data['UpperTailShStDev']
# data['LowvClose_LT_ShStdevTmp'] = (data['Close'].shift(-1) - data['Low']) / data['LowerTailShStDev']

# data['LowvClose_CL_LgStdevTmp'] = (data['Close'].shift(-1) - data['Low']) / data['CandleLengthLgStdev']
# data['LowvClose_CB_LgStdevTmp'] = (data['Close'].shift(-1) - data['Low']) / data['CandleBodyLengthLgStdev']
# data['LowvClose_UT_LgStdevTmp'] = (data['Close'].shift(-1) - data['Low']) / data['UpperTailLgStDev']
# data['LowvClose_LT_LgStdevTmp'] = (data['Close'].shift(-1) - data['Low']) / data['LowerTailLgStDev']

# # Close Standard Dev Tmp (For one hot encoding)
# data['ClosevOpen_CL_ShStdevTmp'] = (data['Open'].shift(-1) - data['Close']) / data['CandleLengthShStdev']
# data['ClosevOpen_CB_ShStdevTmp'] = (data['Open'].shift(-1) - data['Close']) / data['CandleBodyLengthShStdev']
# data['ClosevOpen_UT_ShStdevTmp'] = (data['Open'].shift(-1) - data['Close']) / data['UpperTailShStDev']
# data['ClosevOpen_LT_ShStdevTmp'] = (data['Open'].shift(-1) - data['Close']) / data['LowerTailShStDev']

# data['ClosevOpen_CL_LgStdevTmp'] = (data['Open'].shift(-1) - data['Close']) / data['CandleLengthLgStdev']
# data['ClosevOpen_CB_LgStdevTmp'] = (data['Open'].shift(-1) - data['Close']) / data['CandleBodyLengthLgStdev']
# data['ClosevOpen_UT_LgStdevTmp'] = (data['Open'].shift(-1) - data['Close']) / data['UpperTailLgStDev']
# data['ClosevOpen_LT_LgStdevTmp'] = (data['Open'].shift(-1) - data['Close']) / data['LowerTailLgStDev']

# data['ClosevHigh_CL_ShStdevTmp'] = (data['High'].shift(-1) - data['Close']) / data['CandleLengthShStdev']
# data['ClosevHigh_CB_ShStdevTmp'] = (data['High'].shift(-1) - data['Close']) / data['CandleBodyLengthShStdev']
# data['ClosevHigh_UT_ShStdevTmp'] = (data['High'].shift(-1) - data['Close']) / data['UpperTailShStDev']
# data['ClosevHigh_LT_ShStdevTmp'] = (data['High'].shift(-1) - data['Close']) / data['LowerTailShStDev']

# data['ClosevHigh_CL_LgStdevTmp'] = (data['High'].shift(-1) - data['Close']) / data['CandleLengthLgStdev']
# data['ClosevHigh_CB_LgStdevTmp'] = (data['High'].shift(-1) - data['Close']) / data['CandleBodyLengthLgStdev']
# data['ClosevHigh_UT_LgStdevTmp'] = (data['High'].shift(-1) - data['Close']) / data['UpperTailLgStDev']
# data['ClosevHigh_LT_LgStdevTmp'] = (data['High'].shift(-1) - data['Close']) / data['LowerTailLgStDev']

# data['ClosevLow_CL_ShStdevTmp'] = (data['Low'].shift(-1) - data['Close']) / data['CandleLengthShStdev']
# data['ClosevLow_CB_ShStdevTmp'] = (data['Low'].shift(-1) - data['Close']) / data['CandleBodyLengthShStdev']
# data['ClosevLow_UT_ShStdevTmp'] = (data['Low'].shift(-1) - data['Close']) / data['UpperTailShStDev']
# data['ClosevLow_LT_ShStdevTmp'] = (data['Low'].shift(-1) - data['Close']) / data['LowerTailShStDev']

# data['ClosevLow_CL_LgStdevTmp'] = (data['Low'].shift(-1) - data['Close']) / data['CandleLengthLgStdev']
# data['ClosevLow_CB_LgStdevTmp'] = (data['Low'].shift(-1) - data['Close']) / data['CandleBodyLengthLgStdev']
# data['ClosevLow_UT_LgStdevTmp'] = (data['Low'].shift(-1) - data['Close']) / data['UpperTailLgStDev']
# data['ClosevLow_LT_LgStdevTmp'] = (data['Low'].shift(-1) - data['Close']) / data['LowerTailLgStDev']

# data['ClosevClose_CL_ShStdevTmp'] = (data['Close'].shift(-1) - data['Close']) / data['CandleLengthShStdev']
# data['ClosevClose_CB_ShStdevTmp'] = (data['Close'].shift(-1) - data['Close']) / data['CandleBodyLengthShStdev']
# data['ClosevClose_UT_ShStdevTmp'] = (data['Close'].shift(-1) - data['Close']) / data['UpperTailShStDev']
# data['ClosevClose_LT_ShStdevTmp'] = (data['Close'].shift(-1) - data['Close']) / data['LowerTailShStDev']

# data['ClosevClose_CL_LgStdevTmp'] = (data['Close'].shift(-1) - data['Close']) / data['CandleLengthLgStdev']
# data['ClosevClose_CB_LgStdevTmp'] = (data['Close'].shift(-1) - data['Close']) / data['CandleBodyLengthLgStdev']
# data['ClosevClose_UT_LgStdevTmp'] = (data['Close'].shift(-1) - data['Close']) / data['UpperTailLgStDev']
# data['ClosevClose_LT_LgStdevTmp'] = (data['Close'].shift(-1) - data['Close']) / data['LowerTailLgStDev']


In [0]:
# One Hot Encoding for Output values
# Initial Testing of data['OpenvOpen_CL_ShStdevTmp'] with complete -3 to 3+ one hot encoding (8 Possible Values)
data['OpenvOpen_CL_ShStdevTmp_OneHot<=-3'] = (data['OpenvOpen_CL_ShStdevTmp'] <= -3).astype(int)
data['OpenvOpen_CL_ShStdevTmp_OneHot-3<=-2'] = ((data['OpenvOpen_CL_ShStdevTmp'] > -3) & (data['OpenvOpen_CL_ShStdevTmp'] <= -2)).astype(int)
data['OpenvOpen_CL_ShStdevTmp_OneHot-2<=-1'] = ((data['OpenvOpen_CL_ShStdevTmp'] > -2) & (data['OpenvOpen_CL_ShStdevTmp'] <= -1)).astype(int)
data['OpenvOpen_CL_ShStdevTmp_OneHot-1<=-0'] = ((data['OpenvOpen_CL_ShStdevTmp'] > -1) & (data['OpenvOpen_CL_ShStdevTmp'] <= 0)).astype(int)
data['OpenvOpen_CL_ShStdevTmp_OneHot0<=1'] = ((data['OpenvOpen_CL_ShStdevTmp'] > 0) & (data['OpenvOpen_CL_ShStdevTmp'] <= 1)).astype(int)
data['OpenvOpen_CL_ShStdevTmp_OneHot1<=2'] = ((data['OpenvOpen_CL_ShStdevTmp'] > 1) & (data['OpenvOpen_CL_ShStdevTmp'] <= 2)).astype(int)
data['OpenvOpen_CL_ShStdevTmp_OneHot2<=3'] = ((data['OpenvOpen_CL_ShStdevTmp'] > 2) & (data['OpenvOpen_CL_ShStdevTmp'] <= 3)).astype(int)
data['OpenvOpen_CL_ShStdevTmp_OneHot>3'] = (data['OpenvOpen_CL_ShStdevTmp'] > 3).astype(int)

data = data.drop(columns = "OpenvOpen_CL_ShStdevTmp", errors='ignore')

In [7]:
# Dropping first LgWindow rows removing NaN rolling Window from PreProcessing Data
# Drop Data inputed into Processing

if data.isnull().any().any() == True:
  data = data[LgWindow:]
  print('Rolling Window Removed')

data = data.drop(columns = "Date", errors='ignore')
data = data.drop(columns = "Open", errors='ignore')
data = data.drop(columns = "High", errors='ignore')
data = data.drop(columns = "Low", errors='ignore')
data = data.drop(columns = "Close", errors='ignore')
data = data.drop(columns = "Volume", errors='ignore')

Rolling Window Removed


In [8]:
data.head()

,LengthofCandle,LengthofCandleBody,LowerTailLength,UpperTailLength,CandleLengthShStdev,CandleLengthLgStdev,CandleLengthShVLgStdev,CandleBodyLengthShStdev,CandleBodyLengthLgStdev,CandleBodyLengthShVLgStdev,LowerTailShStDev,LowerTailLgStDev,LowerTailShVLgStdev,UpperTailShStDev,UpperTailLgStDev,UpperTailShVLgStdev,CandleLengthLgPValue,CandleBodyLengthLgPValue,UpperTailLgPValue,LowerTailLgPValue,VolLgPValue,DiffShortO_MA,DiffShortH_MA,DiffShortL_MA,DiffShortC_MA,DiffShortA_MA,DiffLongO_MA,DiffLongH_MA,DiffLongL_MA,DiffLongC_MA,DiffLongA_MA,5DSlpOMA,25DSlpOMA,3-5DSlpOMA,4-6DSlpOMA,5-7DSlpOMA,6-8DSlpOMA,HODvHOD-1,HODvHOD-2,HODvHOD-3,...,Upper-Tailless,Bodyless,GapPrevious1Day,GapPrevious2Day,GapPrevious3Day,GapPrevious4Day,GapPrevious5Day,CloseHigherv1,CloseHigherv2,CloseHigherv3,CloseHigherv4,CloseHigherv5,OpenHigherv1,OpenHigherv2,OpenHigherv3,OpenHigherv4,OpenHigherv5,LodHigherv1,LodHigherv2,LodHigherv3,LodHigherv4,LodHigherv5,HodHigherv1,HodHigherv2,HodHigherv3,HodHigherv4,HodHigherv5,Friday,Monday,Thursday,Tuesday,Wednesday,OpenvOpen_CL_ShStdevTmp_OneHot<=-3,OpenvOpen_CL_ShStdevTmp_OneHot-3<=-2,OpenvOpen_CL_ShStdevTmp_OneHot-2<=-1,OpenvOpen_CL_ShStdevTmp_OneHot-1<=-0,OpenvOpen_CL_ShStdevTmp_OneHot0<=1,OpenvOpen_CL_ShStdevTmp_OneHot1<=2,OpenvOpen_CL_ShStdevTmp_OneHot2<=3,OpenvOpen_CL_ShStdevTmp_OneHot>3
25,2.17,1.52,0.57,0.08,0.509853,1.245427,0.409380,0.812791,0.890405,0.912834,0.419464,0.801755,0.523182,0.343045,0.826184,0.415216,0.335098,0.701354,0.219847,0.325631,0.654023,-1.270,-2.046,-1.666,-2.408,-1.8475,-5.1572,-6.1332,-5.6028,-6.3028,-5.7990,0.014,-0.4396,1.336667,-0.106667,-0.926667,-1.583333,-1.11,-2.13,-4.04,...,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,1,0,1,0,1,1,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0
26,3.42,0.62,1.91,0.89,0.608958,1.252354,0.486251,0.484014,0.890603,0.543468,0.499870,0.813771,0.614263,0.420809,0.778055,0.540847,0.712773,0.314931,0.610727,0.869212,0.984808,-2.260,-1.320,-2.124,-0.840,-1.6360,-6.0016,-5.5252,-6.2284,-5.0572,-5.7031,-0.270,-0.4156,0.400000,1.336667,-0.106667,-0.926667,0.17,-0.94,-1.96,...,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0
27,2.24,1.43,0.01,0.80,0.631625,1.255878,0.502935,0.455005,0.891153,0.510580,0.711920,0.819731,0.868480,0.414126,0.769509,0.538168,0.356604,0.653907,0.552315,0.132007,0.538825,-1.948,-1.842,-1.542,-2.602,-1.9835,-6.4368,-6.6980,-6.2364,-7.5144,-6.7214,-1.222,-0.4748,-0.970000,0.400000,1.336667,-0.106667,-1.62,-1.45,-2.56,...,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
28,1.64,1.06,0.37,0.21,0.683213,1.232141,0.554493,0.455302,0.873980,0.520952,0.753180,0.819731,0.918813,0.384799,0.774798,0.496644,0.189756,0.477189,0.268980,0.248955,0.017156,-1.288,-0.782,-0.148,0.130,-0.5220,-6.0616,-5.8740,-4.7912,-4.6820,-5.3522,-0.700,-0.4152,-0.430000,-0.970000,0.400000,1.336667,0.43,-1.19,-1.02,...,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
29,4.22,3.48,0.29,0.45,1.053290,0.949128,1.109745,1.097643,0.880645,1.246408,0.743236,0.808536,0.919237,0.355007,0.532795,0.666311,0.954200,0.996411,0.411193,0.211866,0.348138,0.448,0.076,-1.406,-2.082,-0.7410,-4.3552,-4.8700,-6.4712,-7.5132,-5.8024,-0.376,-0.3464,-0.763333,-0.430000,-0.970000,0.400000,0.54,0.97,-0.65,...,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0


In [10]:
# multi-class classification with Keras

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# load dataset
dataframe = None
dataframe = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/SpyDailyDF.csv', header=None)
dataset = data.values

training_length = 8
inputData_col = len(dataframe.columns) - training_length -1
outputData_col = len(dataframe.columns) - training_length

inputData = dataset[:,:inputData_col].astype(float)
outputData = dataset[:,outputData_col:]

# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(128, input_dim=inputData_col, activation='relu'))
	model.add(Dense(8, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
	# Save Keras Model
	model.save("/content/drive/My Drive/Colab Notebooks/MelchiorModel.h5")
	print("Saved model to disk")
 
	return model
 
estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, inputData, outputData, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk
Saved model to disk
Baseline: 32.90% (3.25%)


In [0]:
# Export Dataframe to CSV for review. Removing Row and Column Headers
# data.to_csv('your_array.csv', header=False, index=False)
# data.to_csv(r'/content/drive/My Drive/Colab Notebooks/SpyDailyDF.csv', header=False, index=False)

In [11]:
from numpy import loadtxt
from keras.models import load_model

model = load_model("/content/drive/My Drive/Colab Notebooks/MelchiorModel.h5")
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 128)               16896     
_________________________________________________________________
dense_40 (Dense)             (None, 8)                 1032      
Total params: 17,928
Trainable params: 17,928
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Prediction array

TestPredictData = dataset[1,:inputData_col].astype(float)
print(TestPredictData)
print(TestPredictData.shape)
TestPredictData = TestPredictData.reshape(1,-1)
model.predict(TestPredictData, verbose=0)

[ 3.42        0.62        1.91        0.89        0.60895813  1.25235352
  0.48625098  0.48401446  0.89060335  0.54346804  0.49986998  0.81377147
  0.61426334  0.42080875  0.77805484  0.54084715  0.71277259  0.31493122
  0.61072691  0.869212    0.98480765 -2.26       -1.32       -2.124
 -0.84       -1.636      -6.0016     -5.5252     -6.2284     -5.0572
 -5.7031     -0.27       -0.4156      0.4         1.33666667 -0.10666667
 -0.92666667  0.17       -0.94       -1.96       -3.87       -2.78
  0.25       -0.78       -1.04       -3.69        0.16        1.77
 -0.26       -0.01       -2.14       -2.55        2.34        0.96
  1.01       -1.25        0.31       -1.26       -2.29       -2.55
 -5.2        -1.35        0.26       -1.77       -1.52       -3.65
 -4.06        0.83       -0.55       -0.5        -2.76       -1.2
  1.45        0.07        0.12       -2.14       -0.58       -1.08
 -2.46       -2.41       -4.67       -3.11        0.          0.
  0.          0.          0.          

array([[0.24000074, 0.22609338, 0.01496698, 0.05153039, 0.00870557,
        0.00688632, 0.290112  , 0.16170458]], dtype=float32)

In [27]:
# Tuning Batch Size and Number of Epochs

from sklearn.model_selection import GridSearchCV
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(128, input_dim=inputData_col, activation='relu'))
	model.add(Dense(8, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# model = load_model("/content/drive/My Drive/Colab Notebooks/MelchiorModel.h5") 
# Loading a model does not seem to work?

# load dataset
dataframe = None
dataframe = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/SpyDailyDF.csv', header=None)
dataset = data.values

training_length = 8
inputData_col = len(dataframe.columns) - training_length -1
outputData_col = len(dataframe.columns) - training_length

inputData = dataset[:,:inputData_col].astype(float)
outputData = dataset[:,outputData_col:]

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# epochs = [10, 50, 100]

batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 20, 30, 40]

param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(inputData, outputData)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.319973 using {'batch_size': 100, 'epochs': 40}
0.308951 (0.012788) with: {'batch_size': 10, 'epochs': 10}
0.302605 (0.008058) with: {'batch_size': 10, 'epochs': 20}
0.299933 (0.015955) with: {'batch_size': 10, 'epochs': 30}
0.299599 (0.007890) with: {'batch_size': 10, 'epochs': 40}
0.307949 (0.005447) with: {'batch_size': 20, 'epochs': 10}
0.308283 (0.011493) with: {'batch_size': 20, 'epochs': 20}
0.300601 (0.008658) with: {'batch_size': 20, 'epochs': 30}
0.308283 (0.004794) with: {'batch_size': 20, 'epochs': 40}
0.293921 (0.015400) with: {'batch_size': 40, 'epochs': 10}
0.308283 (0.004118) with: {'batch_size': 40, 'epochs': 20}
0.306613 (0.009646) with: {'batch_size': 40, 'epochs': 30}
0.307949 (0.006562) with: {'batch_size': 40, 'epochs': 40}
0.295925 (0.010392) with: {'batch_size': 60, 'epochs': 10}
0.294255 (0.018391) with: {'batch_size': 60, 'epochs': 20}
0.303607 (0.005668) with: {'batch_size': 60, 'epochs': 30}
0.296927 (0.003779) with: {'batch_size': 60, 'epochs': 40}
0

In [0]:
# Tuning the Training Optimization Algorithm
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
	# create model
	model = Sequential()
	model.add(Dense(128, input_dim=inputData_col, activation='relu'))
	model.add(Dense(8, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
dataframe = None
dataframe = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/SpyDailyDF.csv', header=None)
dataset = data.values

training_length = 8
inputData_col = len(dataframe.columns) - training_length -1
outputData_col = len(dataframe.columns) - training_length

inputData = dataset[:,:inputData_col].astype(float)
outputData = dataset[:,outputData_col:]

# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(inputData, outputData)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))